# Codebook

In [1]:
import os, glob, re, subprocess
import pandas as pd

pd.options.display.max_rows = 999

In [2]:
rep = lambda s, n: [ s for i in range(n) ]

data = pd.DataFrame(columns=['org', 'article', 'analysis', 'index', 'level', 'cell', 'code'])

ptrn = os.path.join('.', 'notebooks', '**', '*.html.pdf')
for fn in glob.iglob(ptrn, recursive=True):
    org, article, analysis = fn.split('/')[2:]
    
    proc = subprocess.Popen(['python', './pdfannots/pdfannots.py', fn], stdout=subprocess.PIPE)
    annots = proc.communicate()[0].decode('utf-8')
    codes = re.findall(r'([HML]L)\s\[([^\]]+)\]\s([^\n]+)\n', annots)   
    
    df = pd.DataFrame({
        'org': rep(org, len(codes)),
        'article': rep(article, len(codes)),
        'analysis': rep(analysis, len(codes)),
        'index': [ i for i in range(len(codes)) ],        
        'level': [ c[0].strip().lower() for c in codes ],
        'cell': [ c[1].strip() for c in codes ],
        'code': [ c[2].strip().lower() for c in codes ]
    })
    data = data.append(df)    

data['verb'] = data['code'].apply(lambda row: row.split(' ')[0].lower())

data.head()

,org,article,analysis,index,level,cell,code,verb
0,baltimore-sun-data,2018-voter-registration,01_processing.html.pdf,0,hl,1,turn strings into number.,turn
1,baltimore-sun-data,2018-voter-registration,01_processing.html.pdf,1,ll,1,calculate percentages,calculate
2,baltimore-sun-data,2018-voter-registration,01_processing.html.pdf,2,ll,1,build reusable wrangling workflows,build
3,baltimore-sun-data,2018-voter-registration,01_processing.html.pdf,3,hl,25,"parallel wrangling, perform the same operation...",parallel
4,baltimore-sun-data,2018-voter-registration,01_processing.html.pdf,4,ll,25,combine tables in many files into one table to...,combine


Give me a summary of coding progress so far

In [3]:
summary_stats = [
    len(data['article'].unique()),
    len(data['code'].unique())
]

print('Articles: {}\nCodes: {}'.format(*summary_stats))

Articles: 6
Codes: 170


Display all codes in alphabetical order

In [4]:
data['src'] = data['org'] + '/' + data['article'] + '/' + data['analysis']

data[['level', 'verb', 'code']] \
    .groupby(['level', 'verb', 'code']).count() \
    .sort_values(['level', 'verb', 'code'])

Empty DataFrame
Columns: []
Index: [(hl, analyze, analyze government data), (hl, answer, answer question with data), (hl, compare, compare data from one source against another for comprehensiveness), (hl, explore, explore dataset), (hl, inspect, inspect data for extreme cases), (hl, investigate, investigate differences between geographic populations), (hl, localize, localize a larger story), (hl, look, look for trends in quarterly data), (hl, mung, mung (wrangler) the raw data), (hl, parallel, parallel wrangling, perform the same operations on different tables), (hl, perform, perform the same analysis on a different day), (hl, produce, produce a graphic), (hl, turn, turn strings into number.), (hl, use, use government data), (hl, wrangle, wrangle data from multiple years), (ll, adjust, adjust for inflation), (ll, build, build reusable wrangling workflows), (ll, calculate, calculate average), (ll, calculate, calculate difference), (ll, calculate, calculate donors as proportion), (ll, calculate, calculate monthly average per capita), (ll, calculate, calculate percent), (ll, calculate, calculate percent change), (ll, calculate, calculate percent change in a date range), (ll, calculate, calculate percent change in demographics among different counties), (ll, calculate, calculate percent of total), (ll, calculate, calculate percentages), (ll, calculate, calculate proportion), (ll, calculate, calculate the donation total per candidate and donor.), (ll, calculate, calculate z-score), (ll, canonicalize, canonicalize column names), (ll, combine, combine data with scraping results from a different date), (ll, combine, combine datasets with more data from another source), (ll, combine, combine excel sheets into one table), (ll, combine, combine metadata with results), (ll, combine, combine tables in many files into one table to work with), (ll, combine, combine two tabular data sources), (ll, compare, compare current statistic against all time record), (ll, compare, compare record under different managing bodies), (ll, convert, convert column from date object to string object), (ll, convert, convert column from integer to string object), (ll, convert, convert column to uppercase, possibly perform entity resolution), (ll, convert, convert tk to string), (ll, count, count donations per candidate), (ll, count, count donors that gave to multiple campaigns per candidate), (ll, count, count donors who gave to at least three female candidates), (ll, count, count incidents in date range), (ll, count, count incidents within a date range), (ll, count, count number of complaints filed by facility), (ll, count, count number of entities), (ll, count, count table rows), (ll, count, count the number of inaccurate records), (ll, count, count the number of incidents), (ll, count, count the number of missing records), (ll, count, count total given condition), (ll, count, count total multi-campaign donors by how many different candidates donated to.), (ll, count, count totals comparing by group), (ll, count, count totals given condition), (ll, create, create a lookup table), (ll, create, create a reusable data wrangling subroutine), (ll, create, create a unique key), (ll, create, create data wrangling subroutine), (ll, create, create keys for separate tables and union tables together), (ll, create, create reusable data wrangling subroutine), (ll, determine, determine facilities large discrepancies), (ll, determine, determine how common an issue in the data.), (ll, determine, determine the date range of incidents), (ll, determine, determine the number of excluded incidents), (ll, determine, determine unique donors in table), (ll, drop, drop duplicate column), (ll, drop, drop duplicate rows), (ll, drop, drop duplicate values, keeping the one with richer data.), (ll, drop, drop irrelevant columns), (ll, drop, drop irreverent columns), (ll, drop, drop rows contributing to incomplete data), (ll, drop, drop rows with extraneous values), (ll, duplicate, duplicate